In [11]:
#needed drivers
import time
import pickle
import requests
from bs4 import BeautifulSoup
import os 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pickle
import random

In [2]:
# # if you need to continue your scraping
# #database with all used links
# #(link:used)
# with open("link_used.pkl", "rb") as f:
#         link_used=pickle.load(f)
# # all obtained text in html and their links 
# #(link:text)
# with open("link_tex.pkl", "rb") as b:
#         link_tex=pickle.load(b)

In [18]:
#when you start an have nothing collected
#(link:used)
url="https://linkeddata.overheid.nl/front/portal/document-viewer?id=http%3A%2F%2Flinkeddata.overheid.nl%2Fterms%2Fjurisprudentie%2Fid%2FECLI%3ANL%3AHR%3A1998%3AAA9342&callback=&dates=&fields="
link_used= {}
link_used[url]="N"
#(link:text)
link_tex= {}


In [4]:
def get_data(url,link_used,link_tex):
    #settings for the websdriver
    options = webdriver.ChromeOptions() 
    # dowload location
    options.add_argument("--headless") 
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    time.sleep(1)
    driver.refresh()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    # step 2 find text
    text=soup.find("div", {"class": "tekst"})
    
    if not text:
        link_used[url]="Y"
        return link_used,link_tex
    else:
        #safes link and their text in to the database
        link_tex[url]=str(text)
        # step 3 find links
        links=text.find_all("a", {"class": "popuplink"})
        for a_tag in links:
            #reference text
            link_text = a_tag.get_text(strip=True)
            # link to other court ruling or law
            # Skip empty or whitespace-only links
            if not link_text or link_text.isspace():
                continue
        
            # step 4 add found links to dict with links
            #make the found link in to a usable link
            trans=transform_url(a_tag['href'])
            #checks if not already in the dict
            if trans not in link_used:
                link_used[trans]="N"

    # step 9 set current link as searched .
    link_used[url]="Y"
    driver.close()
    return link_used,link_tex
    

In [5]:
#transform the unicode link to a usable link
def transform_url(url: str) -> str:
    # Extract the id=... part
    m = re.search(r'id=[^&]+', url)
    if not m:
        raise ValueError("No 'id=' parameter found")

    id_param = m.group(0)  # e.g. 'id=http%3A%2F%2F...BJ6879'

    # Build the new URL
    new_url = (
        f"https://linkeddata.overheid.nl/front/portal/document-viewer?"
        f"{id_param}&callback=&dates=&fields="
    )
    return new_url

In [13]:
# if the last used document only leads to an already known court ruling
# goes back to a random used link and  goes to one of the pages that reference to this document and stead of a document it references too
def random_new_url(link_used):
    url =random.choice(list(link_used.keys()))
    m = re.search(r'id=[^&]+', url)
    if not m:
        raise ValueError("No 'id=' parameter found")
    
    id_param = m.group(0)  # e.g. 'id=http%3A%2F%2F...BJ6879'
    
    # Build the new URL
    new_url = (
        f"https://linkeddata.overheid.nl/front/portal/spiegel-lijstweergave?"
        f"{id_param}&callback=&dates=&fields="
    )
    options = webdriver.ChromeOptions() 
    driver = webdriver.Chrome(options=options)
    #login
    driver.get(new_url)
    # time.sleep(1)
    driver.refresh()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    text=soup.find("div", {"class": "results"})
    links=text.find_all("div", {"class": "result-detail"})
    for link in links:
       link_used[transform_url(link.find("a")["href"])]="N"
    return link_used,link.find("a")["href"]




In [14]:
#chooses a new link from the database
def new_url(link_used):
    x = "N"
    first_key="empty"
    #check if unused links
    for k, v in link_used.items():
        if v == x:
            first_key = k
            break
    if first_key=="empty":
        link_used,first_key=random_new_url(link_used)
        print(first_key)

        
    return link_used,transform_url(first_key)

In [ ]:
# picks a url to start
link_used,url=new_url(link_used)
#choose this number as a minum set size
saves=10
max_number=10
for z in range(0,max_number):
    for x in range(0,saves):
        link_used,link_tex = get_data(url,link_used,link_tex)
        #
        link_used,url=new_url(link_used)
    print("size is:" ,len(link_used),len(link_tex))
    with open("link_used.pkl", "wb") as f:
        pickle.dump(link_used, f)
    with open("link_tex.pkl", "wb") as b:
        pickle.dump(link_tex, b)


https://linkeddata.overheid.nl/front/portal/document-viewer?id=http%3A%2F%2Flinkeddata.overheid.nl%2Fterms%2Fjurisprudentie%2Fid%2FECLI%3ANL%3AHR%3A1998%3AAA9342&callback=&dates=&fields=
?id=http%3A%2F%2Flinkeddata.overheid.nl%2Fterms%2Fjurisprudentie%2Fid%2FECLI%3ANL%3APHR%3A1992%3AAC0937&callback=&dates=&fields=&facet.field=%7B%21ex%3Dlink_richting%7Dlink_richting&facet.field=%7B%21ex%3Dobj_type%7Dobj_type&facet.field=%7B%21ex%3Dobj_organisatie_groep%7Dobj_organisatie_groep&facet.field=%7B%21ex%3Dobj_organisatie%7Dobj_organisatie&facet.field=%7B%21ex%3Dlink_type%7Dlink_type&facet.field=%7B%21ex%3Dobj_jaar%7Dobj_jaar
size is: 15 1
